In [1]:
def setup_cam_pop(sim, spike_array, img_w, img_h, w2s=1.6):
    pop_size = img_w*img_h*2
    cell = sim.IF_curr_exp
    params = { 'cm': 0.2,  # nF
               'i_offset': 0.0,
               'tau_m': 10.0,
               'tau_refrac': 2.0,
               'tau_syn_E': 1.,
               'tau_syn_I': 1.,
               'v_reset': -70.0,
               'v_rest': -65.0,
               'v_thresh': -55.4
             }
    cam_pop = sim.Population(pop_size, cell, params,
                             label='camera')
    dmy_pops = []
    dmy_prjs = []
    for i in range(pop_size):
        dmy_pops.append(sim.Population(1, sim.SpikeSourceArray, 
                                       {'spike_times': spike_array[i]},
                                       label='pixel (row, col) = (%d, %d)'%\
                                       (i//img_w, i%img_w)))
        conn = [(0, i, w2s, 1)]
        dmy_prjs.append(sim.Projection(dmy_pops[i], cam_pop,
                                       sim.FromListConnector(conn),
                                       target='excitatory'))
    
    return cam_pop, dmy_pops, dmy_prjs

In [6]:
import numpy as np
from vision.spike_tools.pattern.pattern_generator import *
from pyNN import nest as sim
# from pyNN import spiNNaker as sim
from vision.retina import Retina, dvs_modes, MERGED

#setup simulator
sim.setup(timestep=1., max_delay=100., min_delay=1.)

#Stimulus parameters
#frame rate
fps=30
#frame length
ft=1./fps
#width (pixels)
img_w=32
#height (pixels)
img_h=32
#number of pixels to calculate across
div=4
#corresponding chunk size
chunks_horz=img_w/div
chunks_vert=img_h/div

w2s = 1.6

simulationTime=np.round(2*img_h*1000*ft)#2134. #ms
print 'sim time:',simulationTime

cell = sim.IF_curr_exp
params = { 'cm': 0.2,  # nF
           'i_offset': 0.0,
           'tau_m': 10.0,
           'tau_refrac': 2.0,
           'tau_syn_E': 1.,
           'tau_syn_I': 1.,
           'v_reset': -70.0,
           'v_rest': -65.0,
           'v_thresh': -55.4
         }

#Obtain Spike source array from txt file
SSA=img_spikes_from_to("../inputs/spikes_pol_2.txt", 2*img_w*img_h, 0, 1,simulationTime,0,0)
#SSA=img_spikes_from_to("./moving_bar_spikes/deg_90___dx_1.000000/spikes_pol_MERGED_enc_RATE_thresh_25_hist_99_inh_False___1_frames_at_30fps_16x16_res_spikes.txt",
     #                 2*img_w*img_h, 0, 1,simulationTime,0,0)

#generate input populations    
inputPop, ssa_cam, prj_cam = setup_cam_pop(sim, SSA, 
                                      img_w, img_h, w2s=w2s)

ON_pop = inputPop[0:(img_w*img_h)-1]
OFF_pop= inputPop[img_w*img_h:(2*img_w*img_h)-1]

#setup speed delays
delaysList=range(div-1,-1,-1)
#print delaysList
Delays_medium=np.array(delaysList)*ft*1000
Delays_medium[div-1]=1.
print "Delays used:",Delays_medium

#initialise connection lists
Connections_right=[]
Connections_left=[]
Connections_up=[]
Connections_down=[]

#Specify Populations
rpop=sim.Population(chunks_horz*img_h, cell,  label="right")
lpop=sim.Population(chunks_horz*img_h, cell,  label="left")
upop=sim.Population(chunks_vert*img_w, cell,  label="up")
dpop=sim.Population(chunks_vert*img_w, cell,  label="down")

#declare fixed projection weight
#weight=0.5 #0.0554/4
weight=2./div

#generate relevant connection lists
#left/right connections
for i in range(0,(chunks_horz-1)*img_h):

    for j in range(1,div+1):    
        Connections_right.append([j+(div*i)-1,i,weight,Delays_medium[j-1]])
        Connections_left.append([j+(div*i)-1,i,weight,Delays_medium[div-j]])
        
#up/down connections
for i in range(0,(chunks_vert-1)*img_w):
    
    for j in range(1,div+1):       
        Connections_down.append([(((j-1)*img_w)+i),i,weight,Delays_medium[div-j]])
        #Connections_down.append([(((j-1)*img_w)+(i*img_w*div)),i,weight,Delays_medium[div-j]])
        Connections_up.append([(((j-1)*img_w)+i),i,weight,Delays_medium[j-1]])

#print Connections_down

#setup projections
projectionRightExcited=sim.Projection(ON_pop,rpop,sim.FromListConnector(Connections_right))
projectionLeftExcited=sim.Projection(ON_pop,lpop,sim.FromListConnector(Connections_left))
projectionUpExcited=sim.Projection(ON_pop,upop,sim.FromListConnector(Connections_up))
projectionDownExcited=sim.Projection(ON_pop,dpop,sim.FromListConnector(Connections_down))

#declare simulator requirements
inputPop.record('spikes')
rpop.record('spikes')
lpop.record('spikes')
upop.record('spikes')
dpop.record('spikes')


#print Connections_left
#print Connections_right
print 'OK'



sim time: 2133.0
Delays used: [ 100.           66.66666667   33.33333333    1.        ]
OK


In [3]:
sim.run(simulationTime)
in_spks=inputPop.getSpikes()
right_spks=rpop.getSpikes()
left_spks=lpop.getSpikes()
down_spks=upop.getSpikes()
up_spks=dpop.getSpikes()

sim.end()

print 'OK'


NameError: name 'sim' is not defined

In [4]:
import pylab as plt

from vision.spike_tools.vis import my_imshow, plot_spikes, \
                                   plot_output_spikes, \
                                   imgs_in_T_from_spike_array
plt.figure()              
plot_output_spikes(in_spks,plotter=plt)
plt.figure()               
plot_output_spikes(right_spks,plotter=plt)
plt.figure()                
plot_output_spikes(left_spks,plotter=plt)
plt.figure()     
plot_output_spikes(down_spks,plotter=plt)
plt.figure()     
plot_output_spikes(up_spks,plotter=plt)
plt.xlim([0,simulationTime])
plt.show()

NameError: name 'in_spks' is not defined

In [ ]:
import scipy.misc

numFrames=int(np.ceil(fps*(simulationTime/1000)))

speedMovie=np.zeros([img_h,img_w,numFrames],dtype='uint8')


#Build movie of right speed neuron outputs
for f in range(numFrames):
    
    for i in right_spks:
        ID=i[0]
        time=i[1]/1000
        #pixel ID
        pxID=ID*div
        #y coordinate
        y=np.ceil(pxID/img_w)
        #y=np.ceil(ID/(img_w/div))
        #x coordinate
       # x=pxID/y
        #x=ID-((y-1)*(img_w/div))
        x=pxID-((y-1)*(img_w))
        #light up corresponding speedImage pixels
        #Also light up the preceding ID-div neurons
        if time<=ft*f and time>(ft*(f-1)):
            speedMovie[y,(x-div):x,f]=255
           # speedMovie[y,x:(x+div),f]=255

            #print time
        #set pixels off
        else:
            speedMovie[y,(x-div):x,f]=0
            #speedMovie[y,x:(x+div),f]=0
          #  print x,y
          #  speedMovie[y,x,f]=0


     #write out frame to file
    #fname="./rightMovie/frame_%05d.png"
    fname="./movies/rightMovie/frame_%05d.png"
    scipy.misc.imsave(fname%(f), speedMovie[:,:,f])

#print speedMovie
print numFrames

  

In [ ]:
import scipy.misc

numFrames=int(np.ceil(fps*(simulationTime/1000)))

speedMovie=np.zeros([img_h,img_w,numFrames],dtype='uint8')

count=0

#Build movie of right speed neuron outputs
for f in range(numFrames):
 
    speedMovie[:,count,f]=255
    count=count+1
     #write out frame to file
    fname="./rightMovie/frame_%05d.png"
    scipy.misc.imsave(fname%(f), speedMovie[:,:,f])


In [ ]:
print down_spks

In [41]:
import scipy.misc

numFrames=int(np.ceil(fps*(simulationTime/1000)))

speedMovie=np.zeros([img_h,img_w,numFrames],dtype='uint8')


#Build movie of down speed neuron outputs
for f in range(numFrames):
    
    for i in down_spks:
        ID=i[0]
        time=i[1]/1000
        #pixel ID
        #pxID=ID+(img_w*div)
 
        #x coordinate      
        x=ID-(np.floor(ID/img_w)*img_w)
        
        #y coordinate
        y=np.ceil(ID/img_w)*div        
        
        #print ID,time,x,y
        
        #light up corresponding speedImage pixels
        #Also light up the preceding ID-div neurons
        if time<=ft*f and time>(ft*(f-1)):
            #print y
            speedMovie[y-div:y,x,f]=255
            #speedMovie[y:y+div,x,f]=255
            #print time
        #set pixels off
        else:
            #print x,y,div
            speedMovie[y-div:y,x,f]=0
            #speedMovie[y:y+div,x,f]=0


     #write out frame to file
    fname="./movies/diagonalMovie/frame_%05d.png"
    scipy.misc.imsave(fname%(f), speedMovie[:,:,f])

#print speedMovie
print numFrames

33


In [2]:
from pyNN import spiNNaker as sim